In [1]:
# Trying to figure out the sensitivity of reaction 288 and why my recalculation yields worse results

In [2]:
import os
import sys
import copy
import pickle
import concurrent.futures
import numpy as np
import pandas as pd

import cantera as ct
import rmgpy.chemkin

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def run_simulation(gas, T, P, X):
    # function to run a RCM simulation

    t_end = 1.0  # time in seconds
    gas.TPX = T, P, X

    env = ct.Reservoir(ct.Solution('air.yaml'))
    # env = ct.Reservoir(ct.Solution('air.xml'))
    reactor = ct.IdealGasReactor(gas)
    wall = ct.Wall(reactor, env, A=1.0, velocity=0)
    reactor_net = ct.ReactorNet([reactor])
    # # allegedly faster solving
    # reactor_net.derivative_settings = {"skip-third-bodies": True, "skip-falloff": True}
    # reactor_net.preconditioner = ct.AdaptivePreconditioner()

    times = [0]
    T = [reactor.T]
    P = [reactor.thermo.P]
    X = [reactor.thermo.X]  # mol fractions
    while reactor_net.time < t_end:
        reactor_net.step()

        times.append(reactor_net.time)
        T.append(reactor.T)
        P.append(reactor.thermo.P)
        X.append(reactor.thermo.X)

    return (times, T, P, X)

In [5]:
def get_ignition_delay(times, T, P, X, plot=False, title='', save=''):
    # look for time with largest derivative
    slopes = np.gradient(P, times)
    i = np.argmax(slopes)
    return i, times[i]

In [8]:
def perturb_reaction_ct(rxn, delta):
    # should just be Arrhenius types
    new_kinetics = ct._cantera.Arrhenius(
        np.exp(delta) * rxn.rate.pre_exponential_factor, 
        rxn.rate.temperature_exponent,
        rxn.rate.activation_energy
    )
    rxn.rate = new_kinetics

In [10]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)


cti_path = os.path.join(basedir, 'chem_annotated.cti')
gas = ct.Solution(cti_path)

In [6]:
# Load the experimental conditions
ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/butane_ignition_delay.csv'
df_exp = pd.read_csv(ignition_delay_data)

# slice just table 7, where phi=1.0
table7 = df_exp[df_exp['Table'] == 7]
# Define Initial conditions using experimental data
tau7 = table7['time (ms)'].values.astype(float)  # ignition delay
T7 = table7['T_C'].values  # Temperatures
P7 = table7['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm

In [7]:
# list of starting conditions
# Mixture compositions taken from table 2 of
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
concentrations = []
# for phi = 1
x_diluent = 0.7649
conc_dict = {
    'O2(2)': 0.2038,
    'butane(1)': 0.03135
}

for i in range(0, len(table7)):
    x_N2 = table7['%N2'].values[i] / 100.0 * x_diluent
    x_Ar = table7['%Ar'].values[i] / 100.0 * x_diluent
    x_CO2 = table7['%CO2'].values[i] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    conc_dict['CO2(7)'] = x_CO2
    concentrations.append(conc_dict)

In [13]:
# get the mapping between RMG and Cantera reactions
# Species Diff
common_species = []
RMG2CT_mapping = {}
CT2RMG_mapping = {}
for i, rmg_sp in enumerate(species_list):
    for j, ct_sp in enumerate(gas.species()):
        if str(rmg_sp) == ct_sp.name:
            common_species.append([rmg_sp, ct_sp])
            RMG2CT_mapping[i] = j
            CT2RMG_mapping[j] = i
            break


def ct_matches_rmg_rxn(rmg_rxn, ct_rxn):
    rmg_reactants = set()
    ct_reactants = set()
    for key in ct_rxn.reactants.keys():
        ct_reactants.add(key)
    for reactant in rmg_rxn.reactants:
        rmg_reactants.add(str(reactant))
    if rmg_reactants != ct_reactants:
        return False
    
    rmg_products = set()
    ct_products = set()
    for key in ct_rxn.products.keys():
        ct_products.add(key)
    for product in rmg_rxn.products:
        rmg_products.add(str(product))
    if rmg_products != ct_products:
        return False

    # check duplicates
    if not ct_rxn.duplicate:
        return True
    
    if type(rmg_rxn.kinetics) == rmgpy.kinetics.arrhenius.Arrhenius:
        return np.isclose(rmg_rxn.to_cantera().rate.pre_exponential_factor, ct_rxn.rate.pre_exponential_factor)
    
#     if type(rmg_rxn.kinetics) == rmgpy.kinetics.arrhenius.MultiArrhenius
    
    return False
#     if type(ct_rxn) == list:
#         return False
#     if type(rmg_rxn.to_cantera()) == list:
#         return False
    
#     if type(rmg_rxn.to_cantera() == ct._cantera.FalloffReaction):
#         return False
#     # check the rate
#     return rmg_rxn.to_cantera().rate.pre_exponential_factor == ct_rxn.rate.pre_exponential_factor
    
    
            
# Reaction Diff
common_reactions = []
RMG2CT_rxn_mapping = {}
CT2RMG_rxn_mapping = {}
for i, rmg_rxn in enumerate(reaction_list):
    for j, ct_rxn in enumerate(gas.reactions()):
        if ct_matches_rmg_rxn(rmg_rxn, ct_rxn):
            RMG2CT_rxn_mapping[i] = j
            CT2RMG_rxn_mapping[j] = i
            common_reactions.append([rmg_rxn, ct_rxn])
            break
    else:
        print(f'No match for {reaction_list.index(rmg_rxn)} {rmg_rxn}, {type(rmg_rxn.kinetics)}')
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

No match for 1 O(5) + H2(13) <=> H(14) + OH(15), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 18 HO2(16) + HO2(16) <=> O2(2) + H2O2(17), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 23 OH(15) + H2O2(17) <=> HO2(16) + H2O(8), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 53 O2(2) + C2H3(22) <=> OH(15) + CH2CO(24), <class 'rmgpy.kinetics.arrhenius.MultiPDepArrhenius'>
No match for 60 CH(3) + CH2CHO(21) <=> HCO(19) + C2H3(22), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 83 CH2CHO(21) <=> CO(6) + CH3(18), <class 'rmgpy.kinetics.falloff.Troe'>
No match for 98 OH(15) + CO(6) <=> H(14) + CO2(7), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 110 O2(2) + C2H3(22) <=> CO2(7) + CH3(18), <class 'rmgpy.kinetics.arrhenius.MultiPDepArrhenius'>
No match for 139 O2(2) + C2H3(22) <=> H(14) + CO(6) + CH2O(9), <class 'rmgpy.kinetics.arrhenius.MultiPDepArrhenius'>
No match for 148 O(5) + CH2CHO(21) <=> HCO(19) + CH2O(9

No match for 340 CH3COCH2O2(72) <=> OH(15) + O=C1COC1(917), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 341 [CH2]C(=O)COO(911) <=> OH(15) + O=C1COC1(917), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 342 O=C1COC1(917) <=> CH2O(9) + CH2CO(24), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 343 CH3COCH2O2(72) <=> OH(15) + C=C([O])C[O](920), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 344 [CH2]C(=O)COO(911) <=> OH(15) + C=C([O])C[O](920), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 345 C=C([O])C[O](920) <=> O=C1COC1(917), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 346 O2(2) + O=C1COC1(917) <=> O2(2) + C=C([O])C[O](920), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 347 C=C([O])C[O](920) <=> CH2O(9) + CH2CO(24), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 356 OH(15) + C2H4(11) <=> H(14) + CH3CHO(35), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 360 C2H4O1-2(51) <=> C

No match for 568 CH3(18) + C3H5-A(94) <=> C4H8-1(188), <class 'rmgpy.kinetics.falloff.Troe'>
No match for 569 OH(15) + C4H8O1-3(214) => H2O(8) + CH2O(9) + C3H5-A(94), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 570 H(14) + C4H8O1-3(214) => H2(13) + CH2O(9) + C3H5-A(94), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 571 O(5) + C4H8O1-3(214) => OH(15) + CH2O(9) + C3H5-A(94), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 572 HO2(16) + C4H8O1-3(214) => H2O2(17) + CH2O(9) + C3H5-A(94), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 573 CH3(18) + C4H8O1-3(214) => CH2O(9) + CH4(10) + C3H5-A(94), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 577 C3H5-A(94) + SC4H9(183) <=> C3H6(12) + C4H8-2(189), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 578 C3H5-A(94) + PC4H9(182) <=> C3H6(12) + C4H8-1(188), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 579 C3H5-A(94) + SC4H9(183) <=> C3H6(12) + C4H8-1(188), <class 'rmgpy

No match for 669 IC3H7(93) + [CH2]C[CH]C(748) <=> C3H6(12) + SC4H9(183), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 670 HCO(19) + [CH2]C[CH]C(748) <=> CO(6) + SC4H9(183), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 671 CH3O(31) + [CH2]C[CH]C(748) <=> CH2O(9) + SC4H9(183), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 672 [CH2]C[CH]C(748) + SC4H9(183) <=> C4H8-2(189) + SC4H9(183), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 673 [CH2]C[CH]C(748) + PC4H9(182) <=> C4H8-1(188) + SC4H9(183), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 674 [CH2]C[CH]C(748) + SC4H9(183) <=> C4H8-1(188) + SC4H9(183), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 675 C2H5(33) + [CH2]C[CH]C(748) <=> C2H4(11) + PC4H9(182), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 676 NC3H7(92) + [CH2]C[CH]C(748) <=> C3H6(12) + PC4H9(182), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 677 IC3H7(93) + [CH2]C[CH]C(748) <=> C3

No match for 752 CH2(23) + [CH2]C[CH]C(748) <=> CH3(18) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 753 CH2(23) + C4H8-1(188) <=> CH3(18) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 754 CH2(23) + C4H8-2(189) <=> CH3(18) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 755 C2H(4) + [CH2]C[CH]C(748) <=> C2H2(25) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 756 C2H(4) + C4H8-1(188) <=> C2H2(25) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 757 C2H(4) + C4H8-2(189) <=> C2H2(25) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 758 [CH2]C[CH]C(748) + SC4H9(183) <=> C4H71-3(190) + butane(1), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 759 [CH2]C[CH]C(748) + PC4H9(182) <=> C4H71-3(190) + butane(1), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 760 OH(15) + [CH2]C[CH]C(748) <=> H2O(8) + C4H71-3(190), <cla

...

No match for 1356 [CH2]C(=O)COO(911) + C3KET21(73) <=> CH3COCH2O2(72) + C3KET21(73), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1357 C2H5O(49) + CH3COCH2O2(72) <=> CH3CHO(35) + C3KET21(73), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1358 C2H5O(49) + [CH2]C(=O)COO(911) <=> CH3CHO(35) + C3KET21(73), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1359 CH2O(9) + C3KET21(73) <=> CH3O(31) + [CH2]C(=O)COO(911), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1360 H(14) + [CH2]C[CH]C(748) <=> H2(13) + C4H71-4(191), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1361 C2H2(25) + C2H4(11) <=> [CH2]C=C[CH2](2534), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1362 CH3CO(20) + [CH2]C=C[CH2](2534) <=> CH2CO(24) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1363 CH2CHO(21) + [CH2]C=C[CH2](2534) <=> CH2CO(24) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1364 C2H3(22) + [CH

No match for 1438 [CH2]C(=O)COO(911) + [CH2]C[CH]C(748) <=> C3KET21(73) + C4H71-3(190), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1453 C1OO1(4213) <=> HOCHO(40), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1454 [O]C[O](988) <=> HOCHO(40), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1455 C1OO1(4213) <=> OH(15) + HCO(19), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1456 [O]C[O](988) <=> OH(15) + HCO(19), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1458 HOCHO(40) + C4H6(194) <=> HOCHO(40) + [CH2]C=C[CH2](2534), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1459 HOCHO(40) + C4H6(194) <=> HOCHO(40) + [CH2]C=C[CH2](2534), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1460 HOCHO(40) + C4H6(194) <=> HOCHO(40) + [CH2]C=C[CH2](2534), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No match for 1461 HOCHO(40) + C4H6(194) <=> HOCHO(40) + [CH2]C=C[CH2](2534), <class 'rmgpy.kinetics.chebyshev.Chebyshev'>
No 

No match for 1625 CH3(18) + [CH2]C[CH]C(748) <=> CH4(10) + C4H71-4(191), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1629 CH3O(31) + [CH2]C[O](703) <=> CH3OH(30) + CH2CHO(21), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1630 CH3CO(20) + [CH2]C=C[CH2](2534) <=> CH2CO(24) + C4H71-4(191), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1631 CH2CHO(21) + [CH2]C=C[CH2](2534) <=> CH2CO(24) + C4H71-4(191), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1632 HCO(19) + [CH2]C[O](703) <=> CO(6) + C2H5O(49), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1633 CH3O2(45) + [CH2]C[CH]C(748) <=> CH3O2H(46) + C4H71-4(191), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1634 OCHO(38) + [CH2]C[CH]C(748) <=> HOCHO(40) + C4H71-4(191), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1635 OCHO(38) + C4H8-1(188) <=> HOCHO(40) + C4H71-4(191), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1636 CH3O(31) + [CH2]C[CH]C(74

No match for 1707 H2(13) + SC4H9O2(186) <=> H(14) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1708 C2H6(32) + SC4H9O2(186) <=> C2H5(33) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1709 C3H6(12) + SC4H9O2(186) <=> C3H5-A(94) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1710 C2H4(11) + SC4H9O2(186) <=> C2H3(22) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1711 CH4(10) + SC4H9O2(186) <=> CH3(18) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1712 H2O2(17) + SC4H9O2(186) <=> HO2(16) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1713 OH(15) + SC4H9O(217) <=> SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1714 C4H8OOH2-4(223) + butane(1) <=> SC4H9(183) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1715 C4H8OOH2-4(223) + butane(1) <=> PC4H9(182) + SC4H9O2H(187), 

No match for 1787 C3H6(12) + C4H8OOH2-4(223) <=> C3H5-A(94) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1788 [CH2]C[CH]C(748) + SC4H9O2(186) <=> C4H71-3(190) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1789 [CH2]C[CH]C(748) + C4H8OOH2-4(223) <=> C4H71-3(190) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1790 C4H71-4(191) + SC4H9O2(186) <=> C4H6(194) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1791 C4H71-4(191) + C4H8OOH2-4(223) <=> C4H6(194) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1792 C4H71-3(190) + C4H8OOH2-4(223) <=> C4H6(194) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1793 C3H5O(129) + SC4H9O2(186) <=> C2H3CHO(74) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'>
No match for 1794 C3H5O(129) + C4H8OOH2-4(223) <=> C2H3CHO(74) + SC4H9O2H(187), <class 'rmgpy.kinetics.arrhenius.Arrhenius'

In [12]:
print(reaction_list[288])
print(gas.reactions()[310])

OH(15) + butane(1) <=> H2O(8) + PC4H9(182)
O(5) + butane(1) <=> OH(15) + PC4H9(182)


In [29]:
rxns = set()
for key in CT2RMG_rxn_mapping:
#     print(CT2RMG_rxn_mapping[key])
    rxns.add(CT2RMG_rxn_mapping[key])
print(748 in rxns)

False


In [14]:
RMG2CT_rxn_mapping[288]

309

In [16]:
print(gas.reactions()[309])

OH(15) + butane(1) <=> H2O(8) + PC4H9(182)


In [17]:
print(gas.reactions()[310])

O(5) + butane(1) <=> OH(15) + PC4H9(182)


In [25]:
rmg_reaction_number = 748
ct_reaction_number = RMG2CT_rxn_mapping[rmg_reaction_number]

delta = 0.1

# load a fresh model
gas = ct.Solution(cti_path)
new_reaction = gas.reactions()[ct_reaction_number]
new_kinetics = ct._cantera.Arrhenius(
    np.exp(delta) * new_reaction.rate.pre_exponential_factor,
    new_reaction.rate.temperature_exponent,
    new_reaction.rate.activation_energy
)
new_reaction.rate = new_kinetics
gas.modify_reaction(ct_reaction_number, new_reaction)

# run the simulations at condition #7
i = 7
X = concentrations[i]
t, T, P, X = run_simulation(gas, T7[i], P7[i], X)

index, delay_time = get_ignition_delay(t, T, P, X)
print(delay_time)


KeyError: 748

In [21]:
base_delay = 0.14857534592261107

In [24]:
T7[7]

795